Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population. The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

One factor that proves that he prosecutor's argument is wrong is that the two probabilities are independent. The bloodtype found at the crime scene can be found within 1% of the population and thus there is a 1% chance of the defendant having said blood type. The probability of whether the defendant is guilty or innocent is not solely determined by their blood type. Not having the blood type does not mean there is a 99% chance that the defendant is guilty; it just means they are in the 99% majority of the population that does not have the crime blood type.

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument?

The defender's argument is a case of base rate fallacy. Their statement ignores any potential false positive in the chance that they are innocent. The defender is only taking the probability of the defendant having the blood type into consideration and thinks that it is equal to the probability of the defendant being guilty. This means that the defender is ignoring the probability of the defendant being guilty and the likelihood of the defendant being guilty when it is given that they have the rare blood type.

c. Suppose that forensic analysis tells us that with 98% probability, the blood found at the scene belongs to the perpetrator.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* there is no other evidence.  

    P(B=1|G=1) = 0.98
    P(G=1) = 1/5 = 0.20
    P(B=1) = *no evidence*
    P(G=1|B=1) = (P(B=1|G=1)P(G=1))/P(B=1)
    Since there is no evidence we instead get...
    P(G=1|B=1) = P(B=1|G=1)P(G=1) = 0.98*0.20 = 0.196

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|\hat{X}) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|\hat{X}) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [5]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [6]:
#What is the prior P(Y=ham) ?
ham_prior = 0.5 # the first word is always either 'ham' or 'spam'

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = val / len(word_dictionary)

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [7]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    ham_prior = 0.5
    spam_prior = 0.5
    # Loop over all the words in each message
    for w in m:
        # This try/except handler helps calculate the posterior probabilities
        # as we read through each word when iterating through every message
        try:
            posterior_ham *= ham_prior
            posterior_spam *= spam_prior
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [8]:
#!